In [59]:
import os
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import configparser
import pytz

isLiveApi = 0
epic = 'CS.D.EURUSD.MINI.IP'
resolution = 'DAY'
delta = timedelta(days=20) # historical data can fetch one API call
# Ideal: '2006-01-01T00:00:00' -> '2024-03-29T00:00:00'
start_date_utc = datetime.strptime('2022-11-11T00:00:00', '%Y-%m-%dT%H:%M:%S')
end_date_utc = datetime.strptime('2024-03-29T00:00:00', '%Y-%m-%dT%H:%M:%S')

# Define the UTC timezone
utc = pytz.utc

# Localize the input times to UTC
start_date_utc = utc.localize(start_date_utc)
end_date_utc = utc.localize(end_date_utc)

# Define the target timezone (UTC+7)
target_timezone = pytz.timezone('Asia/Bangkok')  # Example timezone for UTC+7

# Convert the UTC times to the target timezone
start_date_utc_7 = start_date_utc.astimezone(target_timezone)
end_date_utc_7 = end_date_utc.astimezone(target_timezone)

# Format the dates back to strings if needed
start_date_utc_7_str = start_date_utc_7.strftime('%Y-%m-%dT%H:%M:%S')
end_date_utc_7_str = end_date_utc_7.strftime('%Y-%m-%dT%H:%M:%S')

start_date = datetime.strptime(start_date_utc_7_str, '%Y-%m-%dT%H:%M:%S')
end_date = datetime.strptime(end_date_utc_7_str, '%Y-%m-%dT%H:%M:%S')
# List to hold all data frames
data_frames = []

# Read credentials from the configuration file
config = configparser.ConfigParser()
config.read('ig.cfg')

# Function to generate date ranges in 20-day increments
def date_range(start_date, end_date, delta):
    current_date = start_date
    while current_date < end_date:
        yield current_date, min(current_date + delta, end_date)
        current_date += delta


def rename_csv_file_based_on_last_snapshot_time(csv_file_name):
    # Step 1: Read the last line of the CSV file
    with open(csv_file_name, 'r') as file:
        last_line = file.readlines()[-1]
    
    # Step 2: Extract the snapshotTimeUTC value from the last line
    last_line_values = last_line.split(',')
    snapshot_time_utc = last_line_values[1].strip()
    
    # Extract the date part from the snapshotTimeUTC (first 10 characters)
    date_part = snapshot_time_utc[:10]
    
    # Step 3: Construct the new file name
    base_name = os.path.basename(csv_file_name)
    dir_name = os.path.dirname(csv_file_name)
    
    # Extract the start part of the file name (up to the last underscore before the date)
    name_parts = base_name.rsplit('_', 1)
    new_base_name = f"{name_parts[0]}_{date_part}.csv"
    new_csv_file_name = os.path.join(dir_name, new_base_name)
    
    # Step 4: Rename the file
    os.rename(csv_file_name, new_csv_file_name)
    
    return new_csv_file_name
    
if isLiveApi == 0:
    API_KEY = config.get('demo', 'api_key')
    USERNAME = config.get('demo', 'username')
    PASSWORD = config.get('demo', 'password')
    BASE_URL = config.get('demo', 'base_url')
    ACC_ID = config.get('demo', 'acc_number')
else:
    API_KEY = config.get('live', 'api_key')
    USERNAME = config.get('live', 'username')
    PASSWORD = config.get('live', 'password')
    BASE_URL = config.get('live', 'base_url')
    ACC_ID = config.get('live', 'acc_number')

# IG API URLs for demo account
LOGIN_URL = BASE_URL + 'session'
MARKET_URL = BASE_URL + 'markets'
HISTORICAL_DATA_URL = BASE_URL + 'prices/' + epic
# HISTORICAL_DATA_URL = BASE_URL + 'prices/{epic}/{resolution}/{start_date}/{end_date}'

# Authenticate and obtain access token
auth_response = requests.post(LOGIN_URL, json={
    'identifier': USERNAME,
    'password': PASSWORD
}, headers={
    'Content-Type': 'application/json; charset=UTF-8',
    'Accept': 'application/json; charset=UTF-8',
    'X-IG-API-KEY': API_KEY,
    'Version': '3',
    'IG-ACCOUNT-ID': ACC_ID
    }
)

# Extract the access token from the response
access_token = auth_response.json()['oauthToken']['access_token']
token_type = auth_response.json()['oauthToken']['token_type']

# Loop through each date range and fetch data
for start, end in date_range(start_date, end_date, delta):
    start_str = start.strftime('%Y-%m-%dT%H:%M:%S')
    end_str = end.strftime('%Y-%m-%dT%H:%M:%S')

    # Request historical price data for the specified instrument
    history_response = requests.get(HISTORICAL_DATA_URL,
        headers={
            'Content-Type': 'application/json; charset=UTF-8',
            'Accept': 'application/json; charset=UTF-8',
            'Authorization': f"{token_type} {access_token}",
            'Version': '3',
            'X-IG-API-KEY': API_KEY,
            'IG-ACCOUNT-ID': ACC_ID
        },
        params={
            'resolution': resolution,
            'from': start_str,
            'to': end_str,
        }
    )

    # Check for successful response
    if history_response.status_code == 200:
        # Convert JSON response to pandas DataFrame and append to list
        data = history_response.json()
        df = pd.DataFrame(data['prices'])
        data_frames.append(df)
    else:
        print(f"Failed to fetch data from {start_str} to {end_str}. Status code: {history_response.status_code}")
        # end = get_date_days_before(end_str, delta)
        break

# Concatenate all data frames into one
if data_frames:
    all_data = pd.concat(data_frames, ignore_index=True)
    # Write the concatenated data frame to a CSV file
    csv_file_name = f'./getdata/ig/{epic}_{resolution}_{start_date.strftime("%Y-%m-%d")}_{end}.csv'
    all_data.to_csv(csv_file_name, index=False)
    new_csv_file_name = rename_csv_file_based_on_last_snapshot_time(csv_file_name)
    print(f"Data has been successfully written to {new_csv_file_name}")
else:
    print("No data was fetched.")
    


Data has been successfully written to ./getdata/ig/CS.D.EURUSD.MINI.IP_DAY_2022-11-11_2024-03-28.csv


In [60]:
import pandas as pd
import ast

# Function to calculate the average of bid and ask prices
def calculate_avg_price(price_str):
    price_dict = ast.literal_eval(price_str)
    return round(((price_dict['bid'] + price_dict['ask']) / 20000), 7)

# Read the original CSV file
input_file = new_csv_file_name
df = pd.read_csv(input_file)
# Calculate the average prices
df['time'] = pd.to_datetime(df['snapshotTimeUTC'])
df['time'] = df['time'].dt.strftime("%Y-%m-%d 00:00:00")
df['o'] = df['openPrice'].apply(calculate_avg_price)
df['h'] = df['highPrice'].apply(calculate_avg_price)
df['l'] = df['lowPrice'].apply(calculate_avg_price)
df['c'] = df['closePrice'].apply(calculate_avg_price)
df['complete'] = True

# Rename and reorder the columns
df.rename(columns={
    'snapshotTime': 'datetime',
    # 'snapshotTimeUTC': 'time',
    'lastTradedVolume': 'volume'
}, inplace=True)

# Select only the required columns
# df = df[['datetime', 'datetimeUTC', 'o', 'h', 'l', 'c', 'volume']]
df = df[['time', 'o', 'h', 'l', 'c', 'volume', 'complete']]

# Save the transformed DataFrame to a new CSV file
# output_file = './getdata/ig/CS.D.EURUSD.MINI.IP_DAY_2006-01-01_2024-05-15_format.csv'
output_file = f"{input_file}_format.csv"
df.to_csv(output_file, index=False)

print(f"Transformed data saved to {output_file}")

Transformed data saved to ./getdata/ig/CS.D.EURUSD.MINI.IP_DAY_2022-11-11_2024-03-28.csv_format.csv


In [81]:
import os
import pandas as pd
from datetime import datetime

def merge_csv_files(start_merge, end_merge, directory):
    # Convert start and end merge dates to datetime objects
    start_date = datetime.strptime(start_merge, '%Y-%m-%d')
    end_date = datetime.strptime(end_merge, '%Y-%m-%d')

    # List all CSV files in the directory
    all_files = [f for f in os.listdir(directory) if f.endswith('_format.csv')]

    # Filter files based on the date range in the filename
    filtered_files = []
    for file in all_files:
        # Extract dates from the filename
        parts = file.split('_')
        file_start_date_str = parts[4]
        file_end_date_str = parts[5].replace('_format.csv', '')

        try:
            file_start_date = datetime.strptime(file_start_date_str, '%Y-%m-%d')
            file_end_date = datetime.strptime(file_end_date_str, '%Y-%m-%d')
        except ValueError:
            print(f"Filename {file} does not match expected date format and will be skipped.")
            continue

        # Check if the file's date range intersects with the specified range
        if not (file_end_date < start_date or file_start_date > end_date):
            filtered_files.append(file)

    # Sort files by their start date
    filtered_files.sort(key=lambda x: datetime.strptime(x.split('_')[4], '%Y-%m-%d'))

    # Read and concatenate the filtered files
    merged_data = pd.concat([pd.read_csv(os.path.join(directory, f)) for f in filtered_files])

    # Create the output filename
    output_filename = f'CS.D.EURUSD.MINI.IP_DAY_{start_merge}_{end_merge}_format_final.csv'
    output_path = os.path.join(directory, output_filename)

    # Save the merged data to the output file
    merged_data.to_csv(output_path, index=False)
    print(f'Merged file saved as {output_path}')



In [82]:
# Ideal: '2006-01-01' -> '2024-03-29'
merge_csv_files('2006-01-01', '2024-03-29', './getdata/ig')

UnboundLocalError: local variable 'file_start_date' referenced before assignment